In [1]:
import random
from time import time
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
from math import sqrt
import matplotlib.pyplot as plt

%matplotlib inline
tf.reset_default_graph()
CKPTDIR = "./ckptdir-test"

LOAD_MODEL = False


In [2]:
negative_dataset = np.genfromtxt("./negative2.csv", delimiter=',', dtype=["S32", int, "S32",float, float, float, float, float, float, float, float, float, int])
positive_dataset = np.genfromtxt("./positive2.csv", delimiter=',', dtype=["S32", int, "S32",float, float, float, float, float, float, float, float, float, int])

In [3]:
NUM_CLASSES = 2 #  2クラス分類
NUM_STEPS = 500 #  学習回数
LEN_SEQ = 10 # 系列長
SIZE_INPUT = 3 # 入力データ数
NUM_DATA = 2009  # データ数
NUM_TEST = 200 # テスト用のデータ数
SIZE_BATCH = 100 # バッチサイズ
NUM_NODE = 1024  # ノード数
LEARNING_RATE = 0.01  # 学習率

In [4]:
def get_data(dataset):
    """データセットをnparrayに変換する"""
    raw_data = [list(item)[3:6] for item in dataset]
    raw_data = np.array(raw_data)
    return raw_data

In [5]:
positive_data = get_data(positive_dataset)
print (positive_data)[:3]

[[ 0.089 -0.612 -0.76 ]
 [ 0.136 -0.685 -0.7  ]
 [-0.05  -0.521 -0.545]]


In [6]:
def set_matrix(dataset,length):
  tmp = []
  ret = []
  print length
  for i in range(0,length):
    tmp = []
    for j in range(0,10):
      tmp.append(dataset[i*10+j])
    ret.append(np.array(tmp))
  return np.array(ret)

In [7]:

positive_main_data = set_matrix(positive_data,1400)
print positive_main_data[:3]

1400
[[[ 0.089 -0.612 -0.76 ]
  [ 0.136 -0.685 -0.7  ]
  [-0.05  -0.521 -0.545]
  [ 0.131 -0.818 -1.042]
  [ 0.043 -0.685 -0.742]
  [ 0.237 -0.667 -0.506]
  [ 0.467 -0.485 -0.786]
  [-0.028 -0.635 -0.608]
  [ 0.14  -0.845 -1.118]
  [ 0.083 -0.689 -0.586]]

 [[ 0.248 -0.54  -0.79 ]
  [-0.056 -0.62  -0.776]
  [-0.101 -0.598 -0.689]
  [ 0.185 -0.616 -0.711]
  [ 0.45  -0.512 -0.757]
  [ 0.002 -0.506 -0.049]
  [ 0.217 -0.604 -0.732]
  [-0.028 -0.799 -0.917]
  [-0.062 -0.647 -0.753]
  [-0.068 -0.851 -0.962]]

 [[ 0.133 -0.809 -0.947]
  [ 0.097 -0.627 -0.792]
  [ 0.077 -0.7   -0.661]
  [ 0.084 -0.685 -0.689]
  [ 0.116 -0.649 -0.638]
  [-0.02  -0.529 -0.497]
  [ 0.106 -0.587 -0.622]
  [-0.026 -0.681 -0.846]
  [ 0.046 -0.587 -0.654]
  [ 0.008 -0.661 -0.712]]]


In [8]:
def create_label(num,length):
  label = []
  for i in range(0,length):
    label.append(num)

  return np.array(label)

In [9]:
negative_data = get_data(negative_dataset)
#negative_sqrt_data = create_sqrt_data(negative_data)
#neg_len = negative_sqrt_data.size
print negative_data.shape
negative_main_data = set_matrix(negative_data,609)

(6090, 3)
609


In [10]:
print positive_main_data.shape
print negative_main_data.shape

(1400, 10, 3)
(609, 10, 3)


In [11]:
positive_label = create_label(1,1400)
print positive_label.shape
negative_label = create_label(0,609)
print negative_label.shape
print negative_label[:10]

(1400,)
(609,)
[0 0 0 0 0 0 0 0 0 0]


In [12]:
x_data = np.r_[positive_main_data, negative_main_data]
print x_data.shape
#print x_data
y_data = np.r_[positive_label, negative_label]
index_list = np.arange(0, 2009)
np.random.shuffle(index_list)
x_shuffle_data = x_data[index_list]
y_shuffle_data = y_data[index_list]
#print y_data
x_train_data = x_shuffle_data[NUM_TEST:]
y_train_label = y_shuffle_data[NUM_TEST:]
print x_train_data.shape
print y_train_label.shape
x_test_data = x_shuffle_data[:NUM_TEST]
y_test_label = y_shuffle_data[:NUM_TEST]


(2009, 10, 3)
(1809, 10, 3)
(1809,)


In [13]:
x = tf.placeholder(tf.float32, [None, LEN_SEQ, SIZE_INPUT])
t = tf.placeholder(tf.int32, [None])
print x

Tensor("Placeholder:0", shape=(?, 10, 3), dtype=float32)


In [14]:
t_on_hot = tf.one_hot(t, depth=NUM_CLASSES, dtype=tf.float32)

In [15]:
x_transpose = tf.transpose(x, [1, 0, 2])
x_reshape = tf.reshape(x_transpose, [-1, SIZE_INPUT])
x_split = tf.split(x_reshape, LEN_SEQ, 0)

In [16]:
lstm_cell = rnn.BasicLSTMCell(NUM_NODE, forget_bias=1.0)
outputs, states = rnn.static_rnn(lstm_cell, x_split, dtype=tf.float32)

In [17]:
w = tf.Variable(tf.random_normal([NUM_NODE, NUM_CLASSES]))
b = tf.Variable(tf.random_normal([NUM_CLASSES]))
logits = tf.matmul(outputs[-1], w) + b
pred = tf.nn.softmax(logits)

In [18]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=t_on_hot, logits=logits)
loss = tf.reduce_mean(cross_entropy)

In [19]:
optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)
train_step = optimizer.minimize(loss)

In [20]:
correct_prediction = tf.equal(tf.argmax(pred,1), tf.argmax(t_on_hot,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [21]:
loss_train = []
acc_train = []
loss_test = []
acc_test = []

In [22]:
saver = tf.train.Saver()
sess = tf.InteractiveSession()
ckpt = tf.train.get_checkpoint_state(CKPTDIR)
if ckpt:
    # checkpointファイルから最後に保存したモデルへのパスを取得する
    last_model = ckpt.model_checkpoint_path
    print(("load {0}".format(last_model)))
    # 学習済みモデルを読み込む
    saver.restore(sess, last_model)
    LOAD_MODEL = True
else:
  print("initialization")
#sess = tf.Session()
  sess.run(tf.global_variables_initializer())
  start = time()
  i = 0
  for _ in range(NUM_STEPS):
      cycle = int((NUM_DATA-NUM_TEST)/SIZE_BATCH)
      begin = int(SIZE_BATCH * (i % cycle))
      end = int(begin + SIZE_BATCH)
      batch_x, batch_t = x_train_data[begin:end], y_train_label[begin:end]
      i += 1
      sess.run(train_step, feed_dict={x: batch_x, t: batch_t})
      if i % 10 == 0:
          #print begin
          #print end
          loss_, acc_ = sess.run([loss, accuracy], feed_dict={x: batch_x, t: batch_t})
          loss_train.append(loss_)
          acc_train.append(acc_)
          loss_test_, acc_test_ = sess.run([loss, accuracy], feed_dict={x: x_test_data, t: y_test_label})
          loss_test.append(loss_test_)
          acc_test.append(acc_test_)
          print("[%i STEPS] %f sec" % (i, (time() - start)))
          print("[TRAIN] loss : %f, accuracy : %f" %(loss_, acc_))
          print("[TEST loss : %f, accuracy : %f" %(loss_test_, acc_test_))
          
  saver.save(sess, CKPTDIR+"/model")

initialization
[10 STEPS] 0.654960 sec
[TRAIN] loss : 99.984032, accuracy : 0.410000
[TEST loss : 121.157524, accuracy : 0.285000
[20 STEPS] 1.018295 sec
[TRAIN] loss : 52.519337, accuracy : 0.400000
[TEST loss : 62.583149, accuracy : 0.285000
[30 STEPS] 1.364896 sec
[TRAIN] loss : 12.387325, accuracy : 0.270000
[TEST loss : 12.144755, accuracy : 0.285000
[40 STEPS] 1.711979 sec
[TRAIN] loss : 4.720477, accuracy : 0.790000
[TEST loss : 6.406745, accuracy : 0.715000
[50 STEPS] 2.059098 sec
[TRAIN] loss : 3.694108, accuracy : 0.750000
[TEST loss : 4.213658, accuracy : 0.715000
[60 STEPS] 2.405782 sec
[TRAIN] loss : 2.460335, accuracy : 0.260000
[TEST loss : 2.375607, accuracy : 0.285000
[70 STEPS] 2.749063 sec
[TRAIN] loss : 0.819290, accuracy : 0.750000
[TEST loss : 0.930801, accuracy : 0.715000
[80 STEPS] 3.087569 sec
[TRAIN] loss : 0.487893, accuracy : 0.740000
[TEST loss : 0.533898, accuracy : 0.715000
[90 STEPS] 3.425894 sec
[TRAIN] loss : 0.352411, accuracy : 0.710000
[TEST loss : 